In [12]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup 

In [21]:
# scrape from webpage into a df
res = requests.get("https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=next&oldid=942655364")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))

In [22]:
# reset type and column names
df = pd.DataFrame(df[0])
df = df.astype('object')
df.rename(columns={0:'PostCode', 1:'Borough',2:'Neighborhood'}, inplace=True)

In [24]:
# drop first row and rows with no borough assigned
df = df[1:]
df = df[df.Borough != 'Not assigned']

In [25]:
# If a cell has a borough but a Not assigned neighborhood, 
# then the neighborhood will be the same as the borough
neighbor = df[df.Neighborhood == 'Not assigned']

AttributeError: 'DataFrame' object has no attribute 'Neighborhood'

In [26]:
neighbor

,Postal code,Borough,Neighborhood


In [27]:
df.loc[9,"Neighborhood"] = df.loc[9,'Borough']

In [28]:
# group and aggregate to combine neighborhoods for same post code
df.groupby(['PostCode','Borough'], as_index=False, sort=False)

KeyError: 'PostCode'

In [29]:
# join neighborhoods
df= df.groupby(['PostCode','Borough']).aggregate(lambda x: ', '.join(x))

KeyError: 'PostCode'

In [30]:
# add index
df.reset_index()

,index,Postcode,Borough,Neighbourhood,Neighborhood
0,3,M4A,North York,Victoria Village,NaN
1,4,M5A,Downtown Toronto,Harbourfront,NaN
2,5,M6A,North York,Lawrence Heights,NaN
3,6,M6A,North York,Lawrence Manor,NaN
4,7,M7A,Downtown Toronto,Queen's Park,NaN
...,...,...,...,...,...
204,281,M8Z,Etobicoke,Kingsway Park South West,NaN
205,282,M8Z,Etobicoke,Mimico NW,NaN
206,283,M8Z,Etobicoke,The Queensway West,NaN
207,284,M8Z,Etobicoke,Royal York South West,NaN
